In [1]:
import os
import random
import itertools
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
pwd

In [ ]:
base_dir = '../../../dataset/dist/'

train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

train_accepted_dir = os.path.join(train_dir, 'accepted')

train_rejected_dir = os.path.join(train_dir, 'rejected')

valid_accepted_dir = os.path.join(valid_dir, 'accepted')

valid_rejected_dir = os.path.join(valid_dir, 'rejected')

In [ ]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( valid_dir,  batch_size = 20, class_mode = 'binary', target_size = (224, 224))

In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(1, activation='sigmoid'))

In [ ]:
base_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
resnet_history = base_model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 100)

In [ ]:
base_model.save('100epoch.h5')

In [ ]:
pwd

In [ ]:
base_path = '../../../dataset/dist-unbalanced'
SHAPE = (224,224,3)
batch_size = 256

In [ ]:
img = np.random.choice(os.listdir(os.path.join(base_path , "test")))
img_path = os.path.join(base_path, "test", img)
img = cv2.imread(img_path)
img = cv2.resize(img, SHAPE[:2])
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGRA2RGB))
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

In [ ]:
img = image.load_img(img_path, target_size=(224, 224))

In [ ]:
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)

In [ ]:
img_batch.shape

In [ ]:
img_preprocessed = preprocess_input(img_batch)

In [ ]:
img_preprocessed.shape

In [ ]:
model = tf.keras.models.load_model('./models/100epoch.h5')

In [ ]:
model.predict(img_batch)